In [1]:
import pandas as pd
# import sklearn
import pickle


In [2]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import mean_squared_error

In [3]:
#!pip install pyarrow


     |████████████████████████████████| 42.3 MB 36.3 MB/s eta 0:00:01


## Download the data for January and February 2023.


In [6]:
def read_dataframe(filename):
    df = pd.read_parquet(filename)

    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

In [7]:
df_train=read_dataframe('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet')
df_train.info() # January columns are total 19 columns
df_train.head(2)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3009173 entries, 0 to 3066765
Data columns (total 20 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   VendorID               int64         
 1   tpep_pickup_datetime   datetime64[ns]
 2   tpep_dropoff_datetime  datetime64[ns]
 3   passenger_count        float64       
 4   trip_distance          float64       
 5   RatecodeID             float64       
 6   store_and_fwd_flag     object        
 7   PULocationID           object        
 8   DOLocationID           object        
 9   payment_type           int64         
 10  fare_amount            float64       
 11  extra                  float64       
 12  mta_tax                float64       
 13  tip_amount             float64       
 14  tolls_amount           float64       
 15  improvement_surcharge  float64       
 16  total_amount           float64       
 17  congestion_surcharge   float64       
 18  airport_fee           

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee,duration
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.3,1.0,0.5,0.0,0.0,1.0,14.3,2.5,0.0,8.433333
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.9,1.0,0.5,4.0,0.0,1.0,16.9,2.5,0.0,6.316667


In [8]:
df_val=read_dataframe('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet')
df_val.head(2)

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee,duration
0,1,2023-02-01 00:32:53,2023-02-01 00:34:34,2.0,0.3,1.0,N,142,163,2,4.4,3.50,0.5,0.0,0.0,1.0,9.40,2.5,0.00,1.683333
3,1,2023-02-01 00:29:33,2023-02-01 01:01:38,0.0,18.8,1.0,N,132,26,1,70.9,2.25,0.5,0.0,0.0,1.0,74.65,0.0,1.25,32.083333


## Q1.Read the data for January. How many columns are there?

In [5]:
df_train.info() # January columns are total 19 columns
df_train.head(2)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3066766 entries, 0 to 3066765
Data columns (total 19 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   VendorID               int64         
 1   tpep_pickup_datetime   datetime64[ns]
 2   tpep_dropoff_datetime  datetime64[ns]
 3   passenger_count        float64       
 4   trip_distance          float64       
 5   RatecodeID             float64       
 6   store_and_fwd_flag     object        
 7   PULocationID           int64         
 8   DOLocationID           int64         
 9   payment_type           int64         
 10  fare_amount            float64       
 11  extra                  float64       
 12  mta_tax                float64       
 13  tip_amount             float64       
 14  tolls_amount           float64       
 15  improvement_surcharge  float64       
 16  total_amount           float64       
 17  congestion_surcharge   float64       
 18  airport_fee           

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.3,1.0,0.5,0.0,0.0,1.0,14.3,2.5,0.0
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.9,1.0,0.5,4.0,0.0,1.0,16.9,2.5,0.0


## Compute the duration variable. It should contain the duration of a ride in minutes.

In [6]:
df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
df.duration = df.duration.apply(lambda td: td.total_seconds() / 60) 


## Q2. What's the standard deviation of the trips duration in January?

In [7]:
df.duration.describe().round(2)
# std dev = 42.59

count    3066766.00
mean          15.67
std           42.59
min          -29.20
25%            7.12
50%           11.52
75%           18.30
max        10029.18
Name: duration, dtype: float64

In [8]:
df.duration

0           8.433333
1           6.316667
2          12.750000
3           9.616667
4          10.833333
             ...    
3066761    13.983333
3066762    19.450000
3066763    24.516667
3066764    13.000000
3066765    14.400000
Name: duration, Length: 3066766, dtype: float64

## Next, we need to check the distribution of the duration variable. 
There are some outliers. Let's remove them and keep only the records where the duration was between 1 and 60 minutes (inclusive).

In [15]:
total_rec=len(df)
df = df[(df.duration >= 1) & (df.duration <= 60)]

## Q3.What fraction of the records left after you dropped the outliers?

In [16]:
len(df)/total_rec*100 # ~98%

98.1220282212598

## Let's apply one-hot encoding to the pickup and dropoff location IDs. We'll use only these two features for our model.

- Turn the dataframe into a list of dictionaries (remember to re-cast the ids to strings - otherwise it will label encode them)
- Fit a dictionary vectorizer
- Get a feature matrix from it

In [17]:
categorical = ['PULocationID', 'DOLocationID']
df[categorical] = df[categorical].astype(str)



In [18]:
train_dicts = df[categorical].to_dict(orient='records')


In [19]:
dv = DictVectorizer()

X_train = dv.fit_transform(train_dicts)

## Q4.What's the dimensionality of this matrix (number of columns)?

In [20]:
len(dv.feature_names_) # 515 

515

## Now let's use the feature matrix from the previous step to train a model.

- Train a plain linear regression model with default parameters, where duration is the response variable
- Calculate the RMSE of the model on the training data

In [21]:
target = 'duration'
y_train = df[target].values

In [22]:
lr = LinearRegression()
lr.fit(X_train, y_train)

LinearRegression()

In [23]:
y_pred = lr.predict(X_train)

## Q5.What's the RMSE on train?

In [33]:
mean_squared_error(y_train, y_pred, squared=False)
# 7.64 is root_mean_squared_error

7.6492610279057605

## Now let's apply this model to the validation dataset (February 2023).

In [9]:
len(df_train), len(df_val)


(3009173, 2855951)

In [12]:
df_train.describe().round(2)

,VendorID,passenger_count,trip_distance,RatecodeID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee,duration
count,3009173.00,2938068.00,3009173.00,2938068.00,3009173.00,3009173.00,3009173.00,3009173.00,3009173.00,3009173.00,3009173.00,3009173.00,2938068.00,2938068.00,3009173.00
mean,1.73,1.36,3.78,1.42,1.18,17.89,1.55,0.49,3.33,0.49,0.99,26.50,2.29,0.10,14.20
std,0.44,0.90,251.96,6.05,0.51,16.21,1.78,0.09,3.65,1.93,0.17,20.54,0.74,0.35,9.94
min,1.00,0.00,0.00,1.00,0.00,-580.00,-7.50,-0.50,-96.22,-65.00,-1.00,-583.50,-2.50,-1.25,1.00
25%,1.00,1.00,1.09,1.00,1.00,8.60,0.00,0.50,1.00,0.00,1.00,15.45,2.50,0.00,7.22
50%,2.00,1.00,1.80,1.00,1.00,12.80,1.00,0.50,2.74,0.00,1.00,20.13,2.50,0.00,11.55
75%,2.00,1.00,3.30,1.00,1.00,19.80,2.50,0.50,4.14,0.00,1.00,28.30,2.50,0.00,18.18
max,2.00,8.00,258928.15,99.00,4.00,999.00,12.50,53.16,380.80,196.99,1.00,1000.00,2.50,1.25,60.00


In [13]:
df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']

In [14]:
categorical = ['PU_DO'] #'PULocationID', 'DOLocationID']

dv = DictVectorizer()

train_dicts = df_train[categorical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [15]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [16]:
lr = LinearRegression()
lr.fit(X_train, y_train)



LinearRegression()

In [17]:
y_pred = lr.predict(X_val)

## Q6.What's the RMSE on validation?

In [20]:
mean_squared_error(y_val, y_pred,squared=False)

5.198572035205691